In [497]:
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
lone_pine = pd.read_csv(r'C:\Users\wells\OneDrive\Desktop\lone_pine.csv')
crsp_data = pd.read_csv('StockPriceMonthly.csv', encoding='utf8', low_memory=False)

# pre-processing

In [498]:
lone_pine['Ticker'] = lone_pine['Ticker'].astype('str')
def ab(aa):
    if '.' in aa:
        return aa.split('.')[0]
    if '/' in aa:
        return aa.split('/')[0]
    else:
        return aa
lone_pine['Ticker'] = lone_pine['Ticker'].apply(ab)

In [499]:
def abc(aa):
    aa = aa.split('/')
    year = aa[2]
    month = aa[0]
    day = aa[1]
    return year+month
lone_pine['Date'] = lone_pine['Date'].apply(abc)

In [500]:
def abcd(aa):
    return str(aa)[0:6]
crsp_data['date'] = crsp_data['date'].apply(abcd)

In [501]:
add_df = pd.DataFrame(columns=lone_pine.columns)
for index in tqdm_notebook(range(len(lone_pine))):
    each_data = lone_pine.loc[index]
    date = int(each_data['Date'])
    if date % 4 == 0:
        date = date - 12 + 100
    each_data['Date'] = str(date+1)
    each_data['Change'] = 0
    each_data['Change.1'] = 0
    add_df.loc[-1] = list(each_data)
    add_df = add_df.reset_index(drop=True)
    date = int(each_data['Date'])
    each_data['Date'] = str(date+1)
    each_data['Change'] = 0
    each_data['Change.1'] = 0
    add_df.loc[-1] = list(each_data)
    add_df = add_df.reset_index(drop=True)
lone_pine_a = pd.concat([lone_pine, add_df], ignore_index=True)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\wells\AppData\Local\Continuu

In [502]:
def class_(df_col):
    if df_col == 'CL A':
        return 'A'
    elif df_col == 'CL B':
        return 'B'
    elif df_col == 'CL C':
        return 'C'
    elif df_col == 'CL D':
        return 'D'
    else:
        return np.nan
lone_pine_a['Class'] = lone_pine_a['Class'].apply(class_)

In [503]:
def division(df):
    if df['Shares'] == 0:
        return 0
    else:
        return df['Value'] / df['Shares']
lone_pine_a['price'] = lone_pine_a.apply(division, axis=1)

In [504]:
crsp_data.columns = ['PERMNO', 'Date', 'Ticker', 'COMNAM', 'Class', 'DIVAMT', 'PRC', 'VOL']

In [505]:
# def class_1(df_col):
#     if df_col == 'A':
#         return 'A'
#     elif df_col == 'B':
#         return 'B'
#     elif df_col == 'C':
#         return 'C'
#     elif df_col == 'D':
#         return 'D'
#     else:
#         return 0
# crsp_data['Class'] = crsp_data['Class'].apply(class_1)

In [506]:
def to_abs(df_col):
    return abs(df_col)
crsp_data_c = crsp_data[['Date', 'Ticker', 'PRC', 'Class']]
crsp_data_c = crsp_data_c[crsp_data_c['PRC'].notnull()].drop_duplicates(keep='first')
crsp_data_c['PRC'] = crsp_data_c['PRC'].apply(to_abs)

In [507]:
lone_pine_all = lone_pine_a.merge(crsp_data_c, on=['Date', 'Ticker'], how = 'left')

In [508]:
lone_pine_all.drop(['% Port', '% OS', 'Hist'], axis=1, inplace=True)

In [509]:
for name in list(set(list(lone_pine_all['Name']))):
    if len(lone_pine_a[lone_pine_a['Name']==name].reset_index(drop=True)) != len(lone_pine_all[lone_pine_all['Name']==name].reset_index(drop=True)):
        drop_df = lone_pine_all[lone_pine_all['Name']==name].loc[lone_pine_all['Class_x']!=lone_pine_all['Class_y']]
        lone_pine_all.drop(list(drop_df.index), inplace=True)

In [510]:
lone_pine_all = lone_pine_all.sort_values(by='Date').reset_index(drop=True)
df_list = []
for each in tqdm_notebook(lone_pine_all.groupby(by='Name')):
    name, df = each
    df.loc[:, 'last_price'] = df['price'].shift()
    df.loc[:, 'last_PRC'] = df['PRC'].shift()
    df_list.append(df)

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [511]:
lone_pine_all = pd.concat(df_list)

In [512]:
error = lone_pine_all[lone_pine_all['PRC'].isnull()][['Name', 'Ticker', 'PRC', 'Date']].drop_duplicates(keep='first', inplace=False)

In [514]:
test = lone_pine_all
new_error = []
for index in tqdm_notebook(error.index):
    each = error.loc[index]
    df = crsp_data[crsp_data['Ticker']==each['Ticker']]
    try:
        if int(each['Date']) > int(list(df['Date'])[-1]):
            test = test.drop(index)
    except:
        new_error.append(list(each))
lone_pine_all = test

In [519]:
pd.DataFrame(new_error).to_csv('error.csv', index=False)

# revenue

In [536]:
lone_pine_all_t = lone_pine_all
for index in tqdm_notebook(lone_pine_all.index):
    each_data = lone_pine_all.loc[index]
    if each_data['Shares'] == 0 and each_data['Change'] == 0 and each_data['Value'] == 0 and each_data['Change.1'] == 0:
        lone_pine_all_t.drop(index, inplace=True)
lone_pine_all = lone_pine_all_t

In [571]:
def prc_price(df):
    if np.isnan(df['PRC']):
        return df['price']
    else:
        return df['PRC']
lone_pine_all['PRC'] = lone_pine_all.apply(prc_price, axis=1)

In [574]:
def prc_price(df):
    if np.isnan(df['last_PRC']):
        return df['last_price']
    else:
        return df['last_PRC']
lone_pine_all['last_PRC'] = lone_pine_all.apply(prc_price, axis=1)

In [577]:
lone_pine_all.drop(['Class_x', 'price', 'last_price', 'price'], axis=1, inplace=True)
lone_pine_all.columns = ['Name', 'Ticker', 'Shares', 'Shares_Change', 'Value', 'Value_Change', 'Date', 'Price',
                         'Class', 'Last_Price']

In [1088]:
new_share = lone_pine_all[lone_pine_all['Shares']==lone_pine_all['Shares_Change']]
new_share['New_Value'] = new_share['Shares'] * new_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1089]:
new_share.to_csv('new_share.csv', index=False)

In [1090]:
sell_share = lone_pine_all[lone_pine_all['Shares']==0]
sell_share['New_Value'] = sell_share['Shares_Change'] * sell_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1091]:
sell_share.to_csv('sell_share.csv', index=False)

In [1092]:
more_share = lone_pine_all[lone_pine_all['Shares_Change']>0].loc[lone_pine_all['Shares']!=lone_pine_all['Shares_Change']]
more_share['New_Value'] = more_share['Shares'] * more_share['Price']

In [1093]:
more_share.to_csv('more_share.csv', index=False)

In [1094]:
less_share = lone_pine_all[lone_pine_all['Shares_Change']<0].loc[lone_pine_all['Shares']!=0]
less_share['New_Value'] = (less_share['Shares'] + abs(less_share['Shares_Change'])) * less_share['Price']

In [1095]:
less_share.to_csv('less_share.csv', index=False)

In [1105]:
same_share = lone_pine_all[lone_pine_all['Shares_Change']==0]
same_share['New_Value'] = same_share['Shares'] * same_share['Price']

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [1106]:
same_share.to_csv('same_share.csv', index=False)

In [1122]:
total_asset = []
for date in list(set(lone_pine_all['Date'])):
    new_share_d = new_share[new_share['Date']==date]
    sell_share_d = sell_share[sell_share['Date']==date]
    more_share_d = more_share[more_share['Date']==date]
    less_share_d = less_share[less_share['Date']==date]
    same_share_d = same_share[same_share['Date']==date]
    total_asset.append(sum(new_share_d['New_Value'])+abs(sum(sell_share_d['New_Value']))+sum(more_share_d['New_Value'])+sum(less_share_d['New_Value'])+sum(same_share_d['New_Value']))

In [1149]:
revenue = []
for date in list(set(lone_pine_all['Date'])):
    second = lone_pine_all[lone_pine_all['Date']==date]

    sell_shares = second[second['Shares']==0]
    more_shares = second[second['Shares_Change']>0].loc[second['Shares']!=second['Shares_Change']]
    less_shares = second[second['Shares_Change']<0].loc[second['Shares']!=0]
    same_shares = second[second['Shares_Change']==0]

    sell_shares['diff'] = abs(sell_shares['Shares_Change']) * (sell_shares['Price'] - sell_shares['Last_Price'])
    more_shares['diff'] = (more_shares['Shares'] - more_shares['Shares_Change']) * (more_shares['Price'] - more_shares['Last_Price'])
    less_shares['diff'] = (less_shares['Shares'] + abs(less_shares['Shares_Change'])) * (less_shares['Price'] - less_shares['Last_Price'])
    same_shares['diff'] = same_shares['Shares'] * (same_shares['Price'] - same_shares['Last_Price'])
    
    sell_shares = sell_shares[sell_shares['diff'].notnull()]
    more_shares = more_shares[more_shares['diff'].notnull()]
    less_shares = less_shares[less_shares['diff'].notnull()]
    same_shares = same_shares[same_shares['diff'].notnull()]
    
    revenue.append(sum(sell_shares['diff']) + sum(more_shares['diff']) + sum(less_shares['diff']) + sum(same_shares['diff']))

C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\wells\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [1150]:
revenue_df = pd.DataFrame({'date':list(set(lone_pine_all['Date'])),
                          'asset':total_asset,
                          'revenue':revenue})

In [1151]:
revenue_df['return_rate'] = round(revenue_df['revenue'] / revenue_df['asset'] * 100, 3)

In [1153]:
fama = pd.read_csv('fama.csv')
fama['date'] = fama['date'].astype('str')

fama_3 = fama.merge(revenue_df[['date', 'return_rate']], on='date', how='left')

fama_3 = fama_3[fama_3['return_rate'].notnull()]

fama_3['return_rate'] = fama_3['return_rate'] - fama_3['RF']

In [1154]:
fama = pd.read_csv('fama5.csv')
fama['date'] = fama['date'].astype('str')

fama_5 = fama.merge(revenue_df[['date', 'return_rate']], on='date', how='left')

fama_5 = fama_5[fama_5['return_rate'].notnull()]

fama_5['return_rate'] = fama_5['return_rate'] - fama_5['RF']

# lasso

In [1155]:
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

In [1201]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1202]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'train_score', 'test_score', 'e', 'coeff_used'])
alpha = 5
while alpha > 0.001:
    lasso = Lasso(alpha)
    lasso.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = lasso.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = lasso.predict(x_test_std)
    score = r2_score(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(lasso.coef_!=0)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    alpha_df.loc[-1] = [alpha] + list(lasso.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1212]:
alpha_df.sort_values(by='test_score', ascending=False)[0:6]

,alpha,coef1,coef2,coef3,train_score,test_score,e,coeff_used
0,5.000,18.538332,-28.314405,-0.0,0.017308,-58.303747,973.518807,2.0
1,4.999,18.539789,-28.315861,-0.0,0.017309,-58.306749,973.568089,2.0
2,4.998,18.541245,-28.317318,-0.0,0.017309,-58.309751,973.617374,2.0
3,4.997,18.542702,-28.318774,-0.0,0.017310,-58.312754,973.666661,2.0
4,4.996,18.544158,-28.320230,-0.0,0.017310,-58.315756,973.715951,2.0
5,4.995,18.545614,-28.321687,-0.0,0.017310,-58.318759,973.765243,2.0


In [802]:
def pretty_print_linear(coefs, names = None, sort = False):  
    if names == None:  
        names = ["X%s" % x for x in range(len(coefs))]  
    lst = zip(coefs, names)  
    if sort:  
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))  
    return " + ".join("%s * %s" % (round(coef, 3), name)  
                                   for coef, name in lst)  
 
 
 
print('Y = '+ pretty_print_linear(lasso.coef_))

Y = -5.249 * X0 + 21.391 * X1 + 15.716 * X2


fama5

In [1213]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1214]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'train_score', 'test_score', 'e', 'coeff_used'])
alpha = 5
while alpha > 0.001:
    lasso = Lasso(alpha)
    lasso.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = lasso.predict(x_train_std)
    score_train=r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = lasso.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(lasso.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(lasso.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1215]:
alpha_df.sort_values(by='test_score', ascending=False)[0:6]

,alpha,coef1,coef2,coef3,coef4,coef5,train_score,test_score,e,coeff_used
0,5.000,20.204067,-29.259650,-0.0,4.634907,0.0,0.018094,-35.273523,863.294319,3.0
1,4.999,20.205781,-29.261388,-0.0,4.636059,0.0,0.018095,-35.275834,863.349329,3.0
2,4.998,20.207495,-29.263126,-0.0,4.637211,0.0,0.018096,-35.278146,863.404345,3.0
3,4.997,20.209209,-29.264863,-0.0,4.638362,0.0,0.018096,-35.280457,863.459367,3.0
4,4.996,20.210923,-29.266601,-0.0,4.639514,0.0,0.018097,-35.282770,863.514396,3.0
5,4.995,20.212637,-29.268339,-0.0,4.640666,0.0,0.018098,-35.285082,863.569430,3.0


In [792]:
def pretty_print_linear(coefs, names = None, sort = False):  
    if names == None:  
        names = ["X%s" % x for x in range(len(coefs))]  
    lst = zip(coefs, names)  
    if sort:  
        lst = sorted(lst,  key = lambda x:-np.abs(x[0]))  
    return " + ".join("%s * %s" % (round(coef, 3), name)  
                                   for coef, name in lst)  
 
 
 
print('Y = '+ pretty_print_linear(lasso.coef_))

Y = 8.346 * X0 + -0.0 * X1 + 0.0 * X2 + 2.105 * X3 + -0.0 * X4


# Ridge

In [847]:
from sklearn.linear_model import Ridge

In [1216]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1217]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'train_score', 'test_score', 'e', 'coeff_used'])
alpha = 5
while alpha >= 0.001:
    rr = Ridge(alpha)
    rr.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = rr.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = rr.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(rr.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(rr.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1218]:
alpha_df.sort_values(by='test_score', ascending=False)[0:6]

,alpha,coef1,coef2,coef3,train_score,test_score,e,coeff_used
0,5.000,28.801402,-41.378399,-4.473687,0.023725,-131.492857,3367.294562,3.0
1,4.999,28.801801,-41.378837,-4.473806,0.023725,-131.495239,3367.355103,3.0
2,4.998,28.802201,-41.379274,-4.473926,0.023725,-131.497621,3367.415645,3.0
3,4.997,28.802600,-41.379712,-4.474045,0.023725,-131.500003,3367.476190,3.0
4,4.996,28.803000,-41.380149,-4.474165,0.023725,-131.502386,3367.536736,3.0
5,4.995,28.803399,-41.380587,-4.474284,0.023725,-131.504768,3367.597285,3.0


In [1219]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1220]:
alpha_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'train_score', 'test_score', 'e', 'coeff_used'])
alpha = 5
while alpha >= 0.001:
    rr = Ridge(alpha)
    rr.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = rr.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = rr.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(rr.coef_!=0)
    alpha_df.loc[-1] = [alpha] + list(rr.coef_) + [score_train, score, e, coeff_used]
    alpha_df = alpha_df.reset_index(drop=True)
    alpha = alpha - 0.001

In [1221]:
alpha_df.sort_values(by='test_score', ascending=False)[0:6]

,alpha,coef1,coef2,coef3,coef4,coef5,train_score,test_score,e,coeff_used
4998,0.002,2.840842,1.103962,-0.096665,-0.599384,-0.455921,0.165644,-0.022863,222499.312410,5.0
4997,0.003,2.840818,1.103957,-0.096674,-0.599385,-0.455913,0.165644,-0.022863,222499.313160,5.0
4996,0.004,2.840794,1.103953,-0.096682,-0.599386,-0.455905,0.165644,-0.022863,222499.313910,5.0
4995,0.005,2.840770,1.103948,-0.096691,-0.599388,-0.455896,0.165644,-0.022863,222499.314661,5.0
4994,0.006,2.840747,1.103944,-0.096700,-0.599389,-0.455888,0.165644,-0.022863,222499.315411,5.0
4993,0.007,2.840723,1.103940,-0.096709,-0.599390,-0.455880,0.165644,-0.022863,222499.316161,5.0


# SVR

In [857]:
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve

In [1222]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1223]:
c_df = pd.DataFrame(columns=['c', 'coef1', 'coef2', 'coef3', 'train_score', 'test_score', 'e', 'coeff_used'])
c = 100
while c > 1:
    svr_lin = SVR(kernel='linear', C=c, gamma='auto')
    svr_lin.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = svr_lin.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = svr_lin.predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(svr_lin.coef_!=0)
    c_df.loc[-1] = [c] + list(svr_lin.coef_[0]) + [score_train, score, e, coeff_used]
    c_df = c_df.reset_index(drop=True)
    c = c - 1

In [1224]:
c_df.sort_values(by='test_score', ascending=False)[0:6]

,c,coef1,coef2,coef3,train_score,test_score,e,coeff_used
98,2.0,3.730115,0.053653,-0.707161,-0.005909,0.628412,9.097675,3.0
94,6.0,3.768365,0.033870,-0.723266,-0.005891,0.627369,9.123196,3.0
44,56.0,3.768349,0.034044,-0.722981,-0.005891,0.627361,9.123395,3.0
52,48.0,3.768310,0.034028,-0.722978,-0.005891,0.627361,9.123396,3.0
88,12.0,3.768324,0.034015,-0.722982,-0.005891,0.627360,9.123415,3.0
33,67.0,3.768565,0.034060,-0.722995,-0.005891,0.627358,9.123485,3.0


In [1225]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1226]:
c_df = pd.DataFrame(columns=['alpha', 'coef1', 'coef2', 'coef3', 'coef4', 'coef5', 'train_score', 'test_score', 'e', 'coeff_used'])
c = 100
while c > 1:
    svr_lin = SVR(kernel='linear', C=c, gamma='auto')
    svr_lin.fit(x_train_std, y_train['return_rate'].values)
    y_predict_train = svr_lin.predict(x_train_std)
    score_train = r2_score(y_train['return_rate'].values, y_predict_train)
    y_predict = svr_lin.fit(x_train_std, y_train['return_rate'].values).predict(x_test_std)
    score=r2_score(y_test['return_rate'].values, y_predict)
    e = mean_squared_error(y_test['return_rate'].values, y_predict)
    coeff_used = np.sum(svr_lin.coef_!=0)
    c_df.loc[-1] = [c] + list(svr_lin.coef_[0]) + [score_train, score, e, coeff_used]
    c_df = c_df.reset_index(drop=True)
    c = c - 1

In [1227]:
c_df.sort_values(by='test_score', ascending=False)[0:6]

,alpha,coef1,coef2,coef3,coef4,coef5,train_score,test_score,e,coeff_used
28,72.0,3.478645,0.231626,0.168914,-0.539909,-0.539594,-0.006139,0.586846,18.868834,5.0
29,71.0,3.478590,0.231718,0.168880,-0.539857,-0.539602,-0.006139,0.586840,18.869095,5.0
36,64.0,3.478624,0.231696,0.168964,-0.539831,-0.539672,-0.006139,0.586840,18.869122,5.0
63,37.0,3.478637,0.231623,0.168895,-0.539817,-0.539616,-0.006139,0.586838,18.869191,5.0
48,52.0,3.478617,0.231565,0.168768,-0.539760,-0.539594,-0.006139,0.586834,18.869382,5.0
24,76.0,3.478652,0.231583,0.168825,-0.539746,-0.539579,-0.006139,0.586833,18.869431,5.0


# LinearRegression

In [892]:
from sklearn.linear_model import LinearRegression

In [1229]:
x = fama_3[['Mkt-RF', 'SMB', 'HML']]
y = fama_3[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1230]:
lr = LinearRegression(alpha)
lr.fit(x_train_std, y_train['return_rate'].values)
y_predict_train = lr.predict(x_train_std)
score_train = r2_score(y_train['return_rate'].values, y_predict_train)
y_predict = lr.predict(x_test_std)
score=r2_score(y_test['return_rate'].values, y_predict)
e = mean_squared_error(y_test['return_rate'].values, y_predict)
coeff_used = np.sum(lr.coef_!=0)
print('coef:', lr.coef_)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
# list(lr.coef_) + [score_train, score, e, coeff_used]

coef: [ 3.85355434  0.69724029 -0.69264773]
score_train: 0.20782984928838288
score_prediction: -0.021669436693446897
e: 222351.13306960184


In [1231]:
x = fama_5[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML']]
y = fama_5[['return_rate']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

sc = StandardScaler()
sc.fit(x_train)  #可以使用計算得到的均值和方差來對訓練資料做標準化處理
x_train_std = sc.transform(x_train)
x_test_std = sc.transform(x_test)

In [1232]:
lr = LinearRegression(alpha)
lr.fit(x_train_std, y_train['return_rate'].values)
y_predict_train = lr.predict(x_train_std)
score_train = r2_score(y_train['return_rate'].values, y_predict_train)
y_predict = lr.predict(x_test_std)
score=r2_score(y_test['return_rate'].values, y_predict)
e = mean_squared_error(y_test['return_rate'].values, y_predict)
coeff_used = np.sum(lr.coef_!=0)
print('coef:', lr.coef_)
print('score_train:',score_train)
print('score_prediction:',score)
print('e:',e)
# list(lr.coef_) + [score, e, coeff_used]

coef: [ 3.72254478  0.20172483  0.0279679  -0.93653876 -0.34574796]
score_train: 0.6928042989958643
score_prediction: -0.019710340833582807
e: 222380.1131178747
